<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/Gradechecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌱 Grade details for any file from Google drive or uploading directly from your computer.

Note: The csv file should have a column names 'SID' (e.g., passcode for individuals)

## 🌀 Step [1] Install packages

In [ ]:
%%capture
!pip install -U gradio
!pip install -U pydantic typing_extensions

In [ ]:
%%capture
!pip install gradio

## 🌀Step [2] File reading A (Google Drive) or B (Direct uploading)

Run only A or B

In [ ]:
#@markdown (A) Read file (df) from Google Drive
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Ask for folder and file names
folder_name = input("Enter the folder name in your Google Drive where the file is located: ")
file_name = input("Enter the file name (including .csv extension): ")

# Construct the full path and read the CSV file
full_path = f"/content/drive/My Drive/{folder_name}/{file_name}"
try:
    df = pd.read_csv(full_path)
    print("File read successfully. Here's the first few rows of the data:")
    print(df.head())

    # Create a list of column names
    columns = df.columns.tolist()
    print("Column names:", columns)

    # Get the number of columns
    cnum = len(columns)
    print("Number of columns:", cnum)

except Exception as e:
    print(f"An error occurred: {e}")

# Optionally, unmount Google Drive
drive.flush_and_unmount()
print("Google Drive is unmounted.")


In [ ]:
#@markdown (B) Uploading your csv file
import pandas as pd
from google.colab import files

# Function to upload a file and return its path
def upload_file():
    uploaded = files.upload()
    if uploaded:
        for filename in uploaded.keys():
            return filename  # Return the first filename
    return None

# Upload the file and read it into a DataFrame
filename = upload_file()
if filename:
    df = pd.read_csv(filename)
    print("File read successfully. Here's the first few rows of the data:")
    print(df.head())

    # Get the number of columns and the list of column names
    columns = df.columns.tolist()
    cnum = len(columns)

    # Displaying the column details
    print("Number of columns:", cnum)
    print("Column names:", columns)
else:
    print("No file uploaded.")


## 🌀 Step [3] Getting a gradio link

In [ ]:
#@markdown Gradio link
import gradio as gr
import pandas as pd

# Assuming 'cnum' and 'columns' are already defined
# cnum = <number of columns>
# columns = <list of column names>

# Specify the path to your CSV file
csv_file_path = df  # Replace 'gradebook.csv' with your actual file name

# Read the CSV file
# df = pd.read_csv(csv_file_path)

# Ensure 'SID' is string for correct matching
df['SID'] = df['SID'].astype(str)

# Function to get and display student details
def get_student_details(sid):
    student_row = df[df['SID'].str.strip() == sid.strip()]
    if not student_row.empty:
        # Extracting values for separate textboxes
        details = student_row.iloc[0].drop('SID').tolist()
        return details + [""] * (cnum - 1 - len(details))  # Adjust list length based on column count
    else:
        # Return placeholder values if no match is found
        return [""] * (cnum - 1)

# Define the output components dynamically based on the column names
outputs = [gr.Textbox(label=col) for col in columns if col != 'SID']  # Exclude 'SID' from outputs

# Create Gradio interface
iface = gr.Interface(
    fn=get_student_details,
    inputs=gr.Textbox(label="Enter your SID", placeholder="Type your SID here"),
    outputs=outputs,
    title="Phonetics Grade Details (Fall 2023)",
    description="Enter your SID to get your details"
)

iface.launch()
